In [3]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import  LabelEncoder
from sklearn import linear_model
import re


In [4]:
train_data = pd.read_json('<path>/train.json')
test_data = pd.read_json('<path>/test.json')

In [5]:
train_data['num_ingredients'] = train_data['ingredients'].apply(len)
train_data = train_data[train_data['num_ingredients'] > 1]

test_data['num_ingredients'] = test_data['ingredients'].apply(len)
test_data = test_data[test_data['num_ingredients'] > 1]

In [6]:
lemmatizer = WordNetLemmatizer()
def preprocess(ingredients):
    ingredients_text = ' '.join(ingredients)
    ingredients_text = ingredients_text.lower()
    ingredients_text = ingredients_text.replace('-', ' ')
    skip_words = ['water', 'butter', 'cheese', 'salt']
    words = []
    for word in ingredients_text.split():
        if re.findall('[0-9]', word): continue
        if len(word) <= 2: continue
        if '’' in word: continue
        if word in skip_words: continue
        word = lemmatizer.lemmatize(word)
        if len(word) > 0: words.append(word)
    return ' '.join(words)


In [7]:
train_data['x'] = train_data['ingredients'].apply(preprocess)
test_data['x'] = test_data['ingredients'].apply(preprocess)
train_data.head()

,cuisine,id,ingredients,num_ingredients,x
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9,romaine lettuce black olive grape tomato garli...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11,plain flour ground pepper tomato ground black ...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12,egg pepper mayonaise cooking oil green chilies...
3,indian,22213,"[water, vegetable oil, wheat, salt]",4,vegetable oil wheat
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20,black pepper shallot cornflour cayenne pepper ...


In [8]:
count_vect = CountVectorizer(min_df=2 , ngram_range=(1,2))
tfidf_transformer = TfidfTransformer()

In [9]:
final_train=train_data.sample(frac=0.9,random_state=2018)
validation_test=train_data.drop(final_train.index)


In [10]:
X_train = tfidf_transformer.fit_transform(count_vect.fit_transform(final_train["x"]))

final_test = tfidf_transformer.transform(count_vect.transform(test_data["x"]))

validation_train = tfidf_transformer.transform(count_vect.transform(validation_test["x"]))

In [11]:
X_train.shape

(35777, 35600)

In [12]:
final_test.shape

(9936, 35600)

In [13]:
validation_train.shape

(3975, 35600)

In [14]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(final_train['cuisine'].values)
v_train = label_encoder.fit_transform(validation_test['cuisine'].values)
# dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [15]:
logreg = linear_model.LogisticRegression(solver='lbfgs' , C=1000)
logistic = logreg.fit(X_train, y_train)


In [16]:
print(logreg.score(validation_train , v_train))

0.7753459119496855


In [18]:
test_data['cuisine'] = logreg.predict(final_test)

In [22]:
test_data['cuisine'] = label_encoder.inverse_transform(test_data['cuisine'])

/home/kalyan/ds/ds_env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
